In [1]:
import os
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(color_codes=True)
sns.set_style("white")

from plotly.offline import plot
import plotly.graph_objs as go

import sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

import re
import nltk
from nltk.corpus import stopwords
import string
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.porter import *
from string import punctuation

In [2]:
stops = set(stopwords.words("english"))
stemmer = PorterStemmer()
TARGET = 'is_duplicate'
ID = 'id'
NUM_PLACEHOLDER = 'NUMBER'

metrics = list()

In [37]:
def find_unigrams(question):
    word_tokens = question.split()
    word_tokens = [w for w in word_tokens if not w in stops]
    word_tokens = list(map(str.lower, word_tokens))
    return word_tokens

def get_word_frequencies(row):
    return dict(Counter(row).items())

def get_word_freq_proportions(row):
    row_dict_proportion = {a : (b / sum(row.values())) for a , b in row.items()}
    return row_dict_proportion

def get_intersecting_word_counts(row):
    q2_dict = row['q2_freq_dict']
    intersecting_words = {a : b for a , b in row['q1_freq_dict'] if a in q2_dict}
    
    return intersecting_words

def shared_words(row):
    q1_tokens = row['q1_tokens']
    q2_tokens = row['q2_tokens']
    matching_words = [w for w in q1_tokens if w in q2_tokens]
    return matching_words

def shared_prop(row):
    q1_len      = row['q1_len']
    q2_len      = row['q2_len']
    shared_len  = row['shared_len']
    return shared_len / (q1_len + q2_len)

In [4]:
train_set = pd.read_csv(os.getcwd() + '/input/train.csv')
test_set = pd.read_csv(os.getcwd() + '/input/test.csv')

train_set['question1'] = train_set['question1'].fillna('')
train_set['question2'] = train_set['question2'].fillna('')
test_set['question1']  = test_set['question1'].fillna('')
test_set['question2']  = test_set['question2'].fillna('')

In [5]:
train_copy = train_set.copy()

In [6]:
def clean_text(text):
    text = re.sub(r"(What|what|Where|there)\'s", r'\1 is', text)
    text = re.sub(r'(I|i)\'m', r'\1 am', text)
    text = re.sub(r"(do|are|have|did|does|was|is) not", r"\1n't", text)
    text = re.sub(r"can not", r"can't'", text)
    text = re.sub(r"will not", r"won't", text)
    text = re.sub(r"I\'ll", r"I will", text)
    
    text = re.sub(r"(demon)(i)(tization)", r"\1e\3",text)
    text = re.sub("de-monetize", "demonetize",text)
    text = re.sub("19-year-old", "19 year old", text)
    text = re.sub("I\'ve", "I have", text)
    text = re.sub("you\'ve", "you have", text)
    text = re.sub("'re", " are", text)
    text = re.sub("&", "and", text)
    text = re.sub("\s\.\s", "\.", text)
    text = re.sub("“|”", "\"", text)
    
    #Replace multiple occurrences of spaces with a single space
    text = re.sub(r"(\w+)(\s{2})(\w+)", r"\1 \3", text)
    text = re.sub(r"(\w+)(\s{3})(\w+)", r"\1 \3", text)
    text = re.sub(r"(\w+)(\s{2})(\d+)", r"\1 \3", text)
    text = re.sub(r"(\d+)(\s{2})(\w+)", r"\1 \3", text)
    text = re.sub(r"(\.)(\s{2})(\w+)", r"\1 \3", text)
    text = re.sub(r"(\w+)(\s{2})(\.)", r"\1 \3", text)
    text = re.sub(r"(:\)|:\()", r"", text)

    text = re.sub("rupees\snotes|rupees\snote|rupees\scurrency\snotes|rupee\snotes|currency\snotes|Rs notes", "notes", text)
    text = re.sub("currency\snote|rs\snotes|rupee note", "notes", text)
    text = re.sub(r"(Rs\s+)(\d0{2}\sand\s)(Rs\s+)(\d0{3})(\snotes)", r"\2\4\5", text)
    text = re.sub(r"(Rs.\s+)(\d0{2}\sand\s)(Rs.\s+)(\d0{3})(\snotes)", r"\2\4\5", text)
    
    #Replace number abbreviations
    text = re.sub(r"(\d+)(k\s)", r"\g<1>000\s", text)
    return text

In [7]:
train_copy['q1_clean']   = train_copy['question1'].apply(lambda x: clean_text(x))
train_copy['q2_clean']   = train_copy['question2'].apply(lambda x: clean_text(x))

In [8]:
res = train_copy[train_copy['question1'].str.contains(r"\d0{2}\sand\s\d0{3}")]
res.shape

(2003, 8)

In [9]:
res2 = train_copy[train_copy['question1'].str.contains(":")]
res2.shape

(6988, 8)

In [10]:
idx = np.random.choice(res2.index.values)
print(idx)
print(train_copy.loc[idx]['question1'])
print(train_copy.loc[idx]['q1_clean'])

print("----------------------------------------------")
print(train_copy.loc[idx]['question2'])
print(train_copy.loc[idx]['q2_clean'])

118382
Strike the odd one out and explain your rationale: a) jane austen b) george eliot c) charlotte bronte d) homas hardy?
Strike the odd one out and explain your rationale: a) jane austen b) george eliot c) charlotte bronte d) homas hardy?
----------------------------------------------
Strike the odd one out and explain your rationale: a) jane austen b) george eliot c) charlotte bronte d) thomas hardy?
Strike the odd one out and explain your rationale: a) jane austen b) george eliot c) charlotte bronte d) thomas hardy?


In [11]:
train_copy['q1_demonetize']    = train_copy['q1_clean'].str.contains("\d0{2}\sand\s\d0{3}\snotes")
train_copy['q2_demonetize']    = train_copy['q2_clean'].str.contains("\d0{2}\sand\s\d0{3}\snotes")
train_copy['q1_demonetize']    = train_copy['q1_demonetize'].map({True: 1, False: 0})
train_copy['q2_demonetize']    = train_copy['q2_demonetize'].map({True: 1, False: 0})

In [12]:
train_copy['q1_tokens']    = train_copy['q1_clean'].apply(lambda x: find_unigrams(x))
train_copy['q2_tokens']    = train_copy['q2_clean'].apply(lambda x: find_unigrams(x))
train_copy['shared_words'] = train_copy.apply(lambda x: shared_words(x), axis=1)

train_copy['q1_len']       = train_copy['q1_tokens'].apply(len)
train_copy['q2_len']       = train_copy['q2_tokens'].apply(len)

train_copy['shared_len']   = train_copy['shared_words'].apply(len)
train_copy['shared_prop']  = train_copy.apply(lambda x: shared_prop(x), axis=1)

In [38]:
train_copy['q1_freq_dict'] = train_copy['q1_tokens'].apply(lambda row: get_word_frequencies(row))
train_copy['q2_freq_dict'] = train_copy['q2_tokens'].apply(lambda row: get_word_frequencies(row))

train_copy['q1_freq_prop'] = train_copy['q1_freq_dict'].apply(lambda row: get_word_freq_proportions(row))
train_copy['q2_freq_prop'] = train_copy['q2_freq_dict'].apply(lambda row: get_word_freq_proportions(row))

In [36]:
train_copy['intersecting_word_counts'] = train_copy.apply(lambda row: get_intersecting_word_counts(row), axis=1)

ValueError: ('too many values to unpack (expected 2)', 'occurred at index 0')

In [33]:
idx = np.random.choice(train_copy.index.values)
print(idx)
   
print(train_copy.iloc[idx]['q1_freq_dict'])
print("=============================================")
print(train_copy.iloc[idx]['q2_freq_dict'])
print("=============================================")
#print(train_copy.iloc[idx]['intersecting_word_counts'])

228842
Counter({'business': 2, 'terms?': 1, 'layman': 1, 'accelerator': 1, 'incubator': 1, 'what': 1, 'difference': 1, 'vs': 1})
Counter({'incubator': 1, 'what': 1, 'accelerator?': 1, 'difference': 1})
{'incubator': 1, 'what': 1, 'difference': 1}


In [ ]:
X = train_copy[['shared_len', 'shared_prop', 'q1_demonetize', 'q2_demonetize']]
y = train_copy[TARGET]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = RandomForestClassifier(random_state=0)
clf = clf.fit(X_train, y_train)

y_proba = clf.predict_proba(X_test)
log_loss_score = log_loss(y_test, y_proba)
metrics.append(log_loss_score)

In [ ]:
metrics

In [ ]:
train_copy.iloc[idx]['q1_freq_prop']

In [ ]:
# train_corpus = pd.Series(train_set['question1'].tolist())
# len(train_corpus)

In [ ]:
# bow_transformer = CountVectorizer(analyzer='word', ngram_range=(1,2), max_features=256, stop_words='english')
# bow_matrix =  bow_transformer.fit_transform(train_corpus)

# vocab = bow_transformer.get_feature_names()
# bow_matrix = bow_matrix.toarray()

# document_freq = np.sum(bow_matrix, axis=1)
# word_count = np.sum(bow_matrix, axis=0)

# words_count = pd.DataFrame(word_count, columns=['freq'], index=vocab)
# words_count = words_count.sort_values('freq', ascending=False)

In [ ]:
# tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0, max_features=256, stop_words='english')
# tfidf_matrix =  tf.fit_transform(train_corpus)
# dense = tfidf_matrix.todense()
# feature_names = tf.get_feature_names()

In [ ]:
# episode = dense[0].tolist()[0]
# phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0]
# phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)


# for phrase, score in [(feature_names[word_id], score) for (word_id, score) in phrase_scores]:
#    print("{0:<20} {1}".format(phrase, score))

In [ ]:
# gb_qid  = train_set.groupby('qid1').filter(lambda x: len(x) > 1).groupby('qid1')
# duplicate_qid1 = sorted(list(gb_qid.groups))
# len(duplicate_qid1)

In [ ]:
# stats = gb_qid['is_duplicate'].agg({np.sum, np.size})
# only_duplicates = stats.loc[stats['sum'] == stats['size']].sort_values(['size'], ascending=False)
# duplicate_df = train_set.loc[train_set['qid1'].isin(only_duplicates.index)]

In [ ]:
# train_set.loc[train_set['qid1'].isin(duplicate_qid1), 'graph_root'] = 1

# train_set['graph_root'].fillna(0, inplace=True)
# train_set['graph_root'] = train_set['graph_root'].astype(int)

In [ ]:
# for node in only_duplicates.index:
#     group = train_set.loc[train_set['qid1'] == node]
#     group1 = train_set.loc[train_set['qid1'].isin(group['qid2'])]
    
#     if len(group1) > 0:
#         train_set.loc[train_set['qid1'] == node, 'neighbors'] = len(group1)
        
# train_set['neighbors'].fillna(0, inplace=True)
# train_set['neighbors'] = train_set['neighbors'].astype(int)

In [ ]:
#from nltk.book import *